# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [5]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-95820d24bc-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

In [6]:
#STRATEGY: 
#For all the unknown nodes that I want to explore for the first time I will follow the following strategy: 
# - (1) explore all the outcoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (2) explore all the incoming object properties from the node and I will return the property IRI, name and the related object IRI and name 
# - (3) explore all the node data properties

#When I do one of the previous explorative queries, I will write: "Explore **node** by performing (1)" (1) is for the query that explore all the outcoming object properties ... and so on

#I will not do all the previous steps every time, if I find what I need, I will "stop" the exploration strategy

#NOTICE THAT:

#If I have to retrieve the BGP of a node I will do a simple query that selects only the specified node with the 
#specified URI and I retrieve also the label. Obviously the URI that I used is retrieved in the queries that were done
#before that one.



In [7]:
#explore the initial node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q36159 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('obj', 'http://www.wikidata.org/entity/Q12188082'), ('oName', 'James')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('obj', 'http://www.wikidata.org/entity/Q241329'), ('oName', '2006 FIBA World Championship')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for'), ('obj', 'http://www.wikidata.org/entity/Q680221'), ('oName', 'Laureus World Sports Award for Breakthrough of the Year')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('obj', 'http://www.wikidata.org/entity/Q1501719'), ('oName', 'J. Walter Kennedy Citizenship Award')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('obj', 'http://www.wikidata.org/entity/Q163132'), ('oName', 'Akron')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('pName', 'residence'), ('obj', 'http://www.wikidata.org/entity/Q163132'), ('

In [8]:
#explore the initial node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q36159 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q96203815'), ('sName', 'More Than a Vote'), ('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('s', 'http://www.wikidata.org/entity/Q55864389'), ('sName', 'I Promise School'), ('p', 'http://www.wikidata.org/prop/direct/P112'), ('pName', 'founded by')]
[('s', 'http://www.wikidata.org/entity/Q59422166'), ('sName', 'Seattle Kraken'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('s', 'http://www.wikidata.org/entity/Q4896605'), ('sName', 'Best Championship Performance ESPY Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q4896598'), ('sName', 'Best Breakthrough Athlete ESPY Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner')]
[('s', 'http://www.wikidata.org/entity/Q222047'), ('sName', 'NBA Most Valuable Player Award'), ('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'

In [9]:
#explore the initial node by performing (3) 

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q36159 ?p ?obj .
   ?p sc:name ?pName.
   FILTER(isLiteral(?obj))
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pName', 'NUKAT ID'), ('obj', 'n2019024669')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pName', 'NNDB people ID'), ('obj', '524/000023455')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pName', 'CONOR.SI ID'), ('obj', '318966371')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('pName', 'Munzinger Sport number'), ('obj', '01000006363')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID'), ('obj', 'biography/LeBron-James')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('pName', 'Sports-Reference.com Olympic athlete ID'), ('obj', 'ja/lebron-james-1')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('pName', 'name in native language'), ('obj', 'LeBron James')]
[('p', 'http://www.wikidata.org/prop/direct/P1618'), ('pName', 'sport number'), ('obj', '23')]
[('p', 'http://www.wikidata.org/prop/direct/P1618'), ('pName', 'sport number'), ('obj

In [10]:
#explore:
# - every object associated to LeBron James node with incoming object properties that have a name containing the word "basket"
# - every object associated to LeBron James node that has a name containing the word "basket" and that is connected to LeBron James node with an outcoming object property

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q36159 ?p ?obj .
   ?p sc:name ?pName.
   ?obj sc:name ?oName .
   FILTER(REGEX(?pName, "basket") || REGEX(?oName, "basket"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oName', 'basketball player')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q46343203'), ('oName', "men's basketball")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
3


Final query for this task

In [11]:
#retrieve the BGP for basketball player node

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q3665646)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
1


## Task 2
Identify the BGP for basketball.

In [12]:
#from the previous query I see that LeBron James node is associated to the node "basketball" with the property "sport"

Final query for this task

In [13]:
#retrieve the BGP for basketball

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q5372)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


## Task 3
Identify the BGP for basketball team.

In [14]:
#explore:
# - every object associated to LeBron James node with incoming object properties that have a name containing the word "team"
# - every object associated to LeBron James node that have a name containing the word "team" and that is connected to LeBron James node with an outcoming object property

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q36159 ?p ?obj .
   ?p sc:name ?pName.
   ?obj sc:name ?oName .
   FILTER(REGEX(?pName, "team") || REGEX(?oName, "team"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team'), ('obj', 'http://www.wikidata.org/entity/Q169138'), ('oName', 'Miami Heat')]
[('p', 'http://www.wikidata.org/prop/direct/P413'), ('pName', 'position played on team / speciality'), ('obj', 'http://www.wikidata.org/entity/Q308879'), ('oName', 'small forward')]
[('p', 'http://www.wikidata.org/prop/direct/P413'), ('pName', 'position played on team / speciality'), ('obj', 'http://www.wikidata.org/entity/Q462471'), ('oName', 'power forward')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team'), ('obj', 'http://www.wikidata.org/entity/Q121783'), ('oName', 'Los Angeles Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P413'), ('pName', 'position played on team / speciality'), ('obj', 'http://www.wikidata.org/entity/Q212413'), ('oName', 'point guard')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team'), ('obj', 'http://www.wikidat

In [15]:
#explore Los Angeles Lakers node: retrieve only "instance of" object related

queryString = """
SELECT DISTINCT ?inst ?instName
WHERE {
   wd:Q121783 wdt:P31 ?inst . 
   ?inst sc:name ?instName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
[('inst', 'http://www.wikidata.org/entity/Q13393265'), ('instName', 'basketball team')]
1


Final query for this task

In [16]:
#retrieve the BGP for the basketball team

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q13393265)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [17]:
#explore the basketball team node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q13393265 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q51747567'), ('oName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3665646'), ('oName', 'basketball player')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q5137571'), ('oName', 'basketball coach')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q847017'), ('oName', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q12973014'), ('oName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic

In [18]:
#explore the basketball team node by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj 
WHERE {
   wd:Q13393265 ?p ?obj .
   FILTER(isLiteral(?obj)) .
   
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID'), ('obj', 'Basketball-Teams-1')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'Basketball teams')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('pName', 'KBpedia ID'), ('obj', 'BasketballTeam')]
3


In [19]:
#explore the Los Angeles Lakers node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q121783 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q65'), ('oName', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('oName', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pName', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q1781055'), ('oName', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pName', 'home venue'), ('obj', 'http://www.wikidata.org/entity/Q186080'), ('oName', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/dir

In [20]:
#explore the Los Angeles Lakers node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q121783 .
   ?p sc:name ?pName.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17060534'), ('sName', '1947–48 Minneapolis Lakers season'), ('p', 'http://www.wikidata.org/prop/direct/P5138'), ('pName', 'season of club or team')]
[('s', 'http://www.wikidata.org/entity/Q538009'), ('sName', 'Byron Scott'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q553063'), ('sName', 'Steve Blake'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q6932895'), ('sName', 'Steve Hamilton'), ('p', 'http://www.wikidata.org/prop/direct/P647'), ('pName', 'drafted by')]
[('s', 'http://www.wikidata.org/entity/Q2567162'), ('sName', 'Jay Vincent'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q5255286'), ('sName', 'Demetrius Calip'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports t

In [21]:
#explore the Los Angeles Lakers node by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q121783 ?p ?obj .
   FILTER(isLiteral(?obj))
   
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID'), ('obj', 'topic/Los-Angeles-Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('pName', 'Instagram username'), ('obj', 'lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pName', 'Facebook ID'), ('obj', 'losangeleslakers')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('pName', 'VIAF ID'), ('obj', '155971076')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('pName', 'YouTube channel ID'), ('obj', 'UC8CSt-oVqy8pUAoKSApTxQw')]
[('p', 'http://www.wikidata.org/prop/direct/P2397'), ('pName', 'YouTube channel ID'), ('obj', 'UCbNWe0snu7U9H2qioNH2nvg')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('pName', 'Library of Congress authority ID'), ('obj', 'n82271526')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('pName', 'Great Russi

Final query for this task

In [22]:
#from the last queries I note the inception data property of the Los Angeles Lakers node (that is an instance of basktball team) and also the country object property
#so I use them for the final query

queryString = """
SELECT DISTINCT ?team ?teamName ?year 
WHERE {
   #retrieve all the basketball teams with the inception date, their country and their name
   ?team wdt:P31 wd:Q13393265 ;
         wdt:P571 ?inception ;
         wdt:P17 ?country ;
         sc:name ?teamName .

   #binding of the year
   BIND(YEAR(?inception) as ?year) . 
   
   #country name
   ?country sc:name ?countryName .
   
   #filter by country name and year of inception
   FILTER(REGEX(?countryName, "Italy") && (?year < 1945))
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('teamName', 'Nuova Pallacanestro Pavia'), ('year', '1943')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('teamName', 'Fortitudo Pallacanestro Bologna'), ('year', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('teamName', 'Olimpia Milano'), ('year', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('teamName', 'Mens Sana Basket'), ('year', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('teamName', 'Virtus Pallacanestro Bologna'), ('year', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('teamName', 'Pallacanestro Cantù'), ('year', '1936')]
[('team', 'http://www.wikidata.org/entity/Q7127695'), ('teamName', 'Pallacanestro Pavia'), ('year', '1943')]
[('team', 'http://www.wikidata.org/entity/Q3754192'), ('teamName', 'Fulgor Omegna'), ('year', '1937')]
[('team', 'http://www.wikidata.org/entity/Q36572'), ('teamName', 'Reyer Venezia Mestre'), ('year', '1925')]
[('team

## Task 5
Identify the BGP for NBA finals

In [23]:
#from the Los Angelese Lakers node try to retrieve something about the NBA League by using the results of (2) 

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q121783 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q65'), ('oName', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('oName', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pName', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q1781055'), ('oName', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pName', 'home venue'), ('obj', 'http://www.wikidata.org/entity/Q186080'), ('oName', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/dir

In [24]:
#explore the NBA (National Basket Association) node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q155223 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q18536323'), ('oName', 'basketball league')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15986354'), ('oName', 'basketball association')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q238240'), ('oName', 'Eastern Conference')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pName', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q2455323'), ('oName', 'NBA TV')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('obj', 'http://www.wikidata.org/entity/Q810343'), ('oName', 'Basketball Association of America')]
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('pName', 'season ends'), ('obj', 'http://www.wikidata.org/entity/Q118'), ('oName', 'April')]
[('p', 'http://www.wikidata.org/p

In [25]:
#explore the NBA (National Basket Association) node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q155223 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q7352141'), ('sName', 'Robertson v. National Basketball Association'), ('p', 'http://www.wikidata.org/prop/direct/P1591'), ('pName', 'defendant')]
[('s', 'http://www.wikidata.org/entity/Q10950388'), ('sName', 'Template:NBA seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q106574439'), ('sName', 'Matthew Maurer'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('s', 'http://www.wikidata.org/entity/Q22005771'), ('sName', 'Bill Kennedy'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q2566198'), ('sName', 'Leon Wood'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('s', 'http://www.wikidata.org/entity/Q3638842'), ('sName', 'Bernie Fryer'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('s', 'http://www.

In [26]:
#from the previous query try to filter out something about the NBA Finals

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q155223 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   
   FILTER(REGEX(?sName, "NBA") || REGEX(?pName, "NBA"))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q10950388'), ('sName', 'Template:NBA seasons'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q60485339'), ('sName', 'NBA Expansion Draft'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('s', 'http://www.wikidata.org/entity/Q93735'), ('sName', '1961 NBA Expansion Draft'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('s', 'http://www.wikidata.org/entity/Q2455323'), ('sName', 'NBA TV'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('s', 'http://www.wikidata.org/entity/Q6952699'), ('sName', 'NBA Store'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('s', 'http://www.wikidata.org/entity/Q2455323'), ('sName', 'NBA TV'), ('p', 'http://www.wikidata.org/prop/direct/P137'), ('pName', 'operator')]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '

In [27]:
#explore the NBA node by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q155223 ?p ?obj .
   FILTER(isLiteral(?obj))
   
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pName', 'number of participants'), ('obj', '30')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pName', 'CANTIC ID'), ('obj', 'a11667941')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pName', 'Encyclopædia Britannica Online ID'), ('obj', 'topic/National-Basketball-Association')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBA')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBACanada')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBAFantasy')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBAOfficial')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBAPR')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username'), ('obj', 'NBAcom')]
[('p', 'htt

In [28]:
#from the exploration (2) of the NBA node, try to find something about the "organizer" property 

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s wdt:P664 wd:Q155223 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .

}
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/P348'), ('pName', 'software version identifier')]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/P4669'), ('pName', 'tabular software version')]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/P747'), ('pName', 'has edition or translation')]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/Q1002728'), ('pName', 'Fonsi Nieto')]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/Q100326534'), ('pName', "Al Stewart's albums in chronological order")]
[('s', 'http://www.wikidata.org/entity/Q600389'), ('sName', '1990–91 NBA season'), ('p', 'http://www.wikidata.org/entity/Q100327667

In [29]:
#from the previous query try to filter out all the NBA seasons

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q155223 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
   
   FILTER(!REGEX(?sName, "season") && REGEX(?sName, "NBA"))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q60485339'), ('sName', 'NBA Expansion Draft'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('s', 'http://www.wikidata.org/entity/Q93735'), ('sName', '1961 NBA Expansion Draft'), ('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('s', 'http://www.wikidata.org/entity/Q2455323'), ('sName', 'NBA TV'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('s', 'http://www.wikidata.org/entity/Q6952699'), ('sName', 'NBA Store'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('s', 'http://www.wikidata.org/entity/Q2455323'), ('sName', 'NBA TV'), ('p', 'http://www.wikidata.org/prop/direct/P137'), ('pName', 'operator')]
[('s', 'http://www.wikidata.org/entity/Q15719374'), ('sName', '2013 NBA Summer League'), ('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pName', 'sports season of league or competition')]
[('s', 'http://www.wikidata.org/entity/

In [30]:
#explore a NBA World Championship Series node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q4566866 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q128069'), ('oName', 'Syracuse')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('obj', 'http://www.wikidata.org/entity/Q36091'), ('oName', 'Minneapolis')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('oName', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q842375'), ('oName', 'NBA Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName',

Final query for this task

In [31]:
#from the previous query retrieve the BGP for the NBA Finals

queryString = """
SELECT DISTINCT ?uri ?name
WHERE {
   ?uri ?p ?obj .
   # get the label
   ?uri sc:name ?name .
   FILTER(?uri = wd:Q842375)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q842375'), ('name', 'NBA Finals')]
1


## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [32]:
#retrieve all the teams where Kevin Durant and LeBron James played 

queryString = """
SELECT DISTINCT ?team ?teamName ?playerName 
WHERE {
   #retrieve all the teams 
   ?team wdt:P31 wd:Q13393265 . 
   
   #retrieve all the basketball players and their team
   ?player wdt:P106 wd:Q3665646 ;
           wdt:P54 ?team . 
           
   #obtaine player and team name
   ?player sc:name ?playerName . 
   ?team sc:name ?teamName . 
   
   #consider only LeBron James and Kevin Durant players
   FILTER(REGEX(?playerName,"LeBron James") || REGEX(?playerName,"Kevin Durant")) 
}
ORDER BY ?playerName
LIMIT 100

"""

print("Results")
x=run_query(queryString)


Results
[('team', 'http://www.wikidata.org/entity/Q180950'), ('teamName', 'Oklahoma City Thunder'), ('playerName', 'Kevin Durant')]
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics'), ('playerName', 'Kevin Durant')]
[('team', 'http://www.wikidata.org/entity/Q572134'), ('teamName', 'Brooklyn Nets'), ('playerName', 'Kevin Durant')]
[('team', 'http://www.wikidata.org/entity/Q7707830'), ('teamName', "Texas Longhorns men's basketball"), ('playerName', 'Kevin Durant')]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors'), ('playerName', 'Kevin Durant')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat'), ('playerName', 'LeBron James')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('playerName', 'LeBron James')]
[('team', 'http://www.wikidata.org/entity/Q162990'), ('teamName', 'Cleveland Cavaliers'), ('playerName', 'LeBron James')]
8


In [33]:
#explore the NBA Finals node by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q842375 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)





Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1366722'), ('oName', 'final')]
[('p', 'http://www.wikidata.org/prop/direct/P2922'), ('pName', 'month of the year'), ('obj', 'http://www.wikidata.org/entity/Q118'), ('oName', 'April')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15275719'), ('oName', 'recurring event')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q14335327'), ('oName', 'Template:Infobox basketball final')]
[('p', 'http://www.wikidata.org/pr

In [34]:
#explore the NBA Finals node by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q842375 .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q24264147'), ('sName', '2016 NBA Finals'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q29096468'), ('sName', '2017 NBA Finals'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q14335327'), ('sName', 'Template:Infobox basketball final'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('s', 'http://www.wikidata.org/entity/Q8663431'), ('sName', 'Category:National Basketball Association Finals'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q4566514'), ('sName', '1949 BAA World Championship Series'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q4566866'), ('sName', '1950 NBA World Championship Series'), ('p', 'http://w

In [35]:
#explore the NBA Finals node by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj 
WHERE {
   wd:Q842375 ?p ?obj .
   FILTER(isLiteral(?obj))
   
   ?p sc:name ?pName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('pName', 'official name'), ('obj', 'National Basketball Association World Championship Series')]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('pName', 'official name'), ('obj', 'Basketball Association of America World Championship Series')]
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('pName', 'short name'), ('obj', 'The Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID'), ('obj', 'NBA-Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('obj', 'NBA Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception'), ('obj', '1947-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/03m5x4')]
7


In [36]:
#explore one of the node that is istance of NBA Finals (2017 NBA Finals in this case) by performing (1)

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q29096468 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q24264147'), ('oName', '2016 NBA Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q842375'), ('oName', 'NBA Finals')]
4


In [37]:
#explore one of the node that is istance of NBA Finals (2017 NBA Finals in this case) by performing (2)

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q29096468 .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q24264147'), ('sName', '2016 NBA Finals'), ('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by')]
1


In [38]:
#explore one of the node that is istance of NBA Finals (2017 NBA Finals in this case) by performing (3)

queryString = """
SELECT DISTINCT ?p ?pName ?obj 
WHERE {
   wd:Q29096468 ?p ?obj .
   FILTER(isLiteral(?obj))
   
   ?p sc:name ?pName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pName', 'Google Knowledge Graph ID'), ('obj', '/g/11c7w124q9')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('pName', 'point in time'), ('obj', '2017-01-01T00:00:00Z')]
2


In [39]:
#explore again the Los Angeles Lakers Node by perfoming (2) 

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q121783 .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?s sc:name ?sName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q17060534'), ('sName', '1947–48 Minneapolis Lakers season'), ('p', 'http://www.wikidata.org/prop/direct/P5138'), ('pName', 'season of club or team')]
[('s', 'http://www.wikidata.org/entity/Q538009'), ('sName', 'Byron Scott'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q553063'), ('sName', 'Steve Blake'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q6932895'), ('sName', 'Steve Hamilton'), ('p', 'http://www.wikidata.org/prop/direct/P647'), ('pName', 'drafted by')]
[('s', 'http://www.wikidata.org/entity/Q2567162'), ('sName', 'Jay Vincent'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports team')]
[('s', 'http://www.wikidata.org/entity/Q5255286'), ('sName', 'Demetrius Calip'), ('p', 'http://www.wikidata.org/prop/direct/P54'), ('pName', 'member of sports t

Final query for this task

In [40]:
#from the previous query use the "winner" property and perform the final query

queryString = """
SELECT ?team ?teamName (COUNT(DISTINCT ?finals) as ?nWins)
WHERE {
   #select all the team nodes
   ?team wdt:P31 wd:Q13393265 .

   #select all the players that play in a team
   ?player wdt:P106 wd:Q3665646 ;
           wdt:P54 ?team . 
           
   #retrieve the team and player names
   ?player sc:name ?playerName . 
   ?team sc:name ?teamName . 
   
   #retrieve all the finals won by these teams
   ?finals wdt:P31 wd:Q842375 ; 
           wdt:P1346 ?team .
   FILTER(REGEX(?playerName,"LeBron James") || REGEX(?playerName,"Kevin Durant")) 
}
GROUP BY ?team ?teamName
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('team', 'http://www.wikidata.org/entity/Q235326'), ('teamName', 'Seattle SuperSonics'), ('nWins', '1')]
[('team', 'http://www.wikidata.org/entity/Q121783'), ('teamName', 'Los Angeles Lakers'), ('nWins', '16')]
[('team', 'http://www.wikidata.org/entity/Q169138'), ('teamName', 'Miami Heat'), ('nWins', '2')]
[('team', 'http://www.wikidata.org/entity/Q157376'), ('teamName', 'Golden State Warriors'), ('nWins', '4')]
4


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [41]:
#from the LeBron James node I retrieve: P27 property that is "country of citizenship", I explore the node associate with this
#property to LeBron James node by performing (1)


queryString = """
SELECT DISTINCT ?p ?pName ?obj ?oName
WHERE {
   wd:Q30 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName. 
   FILTER(REGEX(?pName, "continent"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent'), ('obj', 'http://www.wikidata.org/entity/Q49'), ('oName', 'North America')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent'), ('obj', 'http://www.wikidata.org/entity/Q538'), ('oName', 'Insular Oceania')]
2


Final query for this task

In [42]:
#final query

queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT ?player) as ?numberOfPlayers)
WHERE {
   #get all the team in the NBA 
   ?team wdt:P31 wd:Q13393265 ;
         wdt:P118 wd:Q155223 .
         
   #get all the players in these teams
   ?player wdt:P106 wd:Q3665646 ;
           wdt:P54 ?team ; 
           wdt:P27 ?country .
           
   #get the continent of the country and the name of the continent
   ?country wdt:P30 ?continent .
   ?continent sc:name ?continentName .
}
GROUP BY ?continent ?continentName 
LIMIT 100 
"""

print("Results")
x=run_query(queryString)


Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('numberOfPlayers', '300')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('numberOfPlayers', '3770')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('numberOfPlayers', '60')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('numberOfPlayers', '73')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('numberOfPlayers', '17')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('numberOfPlayers', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('numberOfPlayers', '3820')]
7


## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

Final query for this task

In [43]:
#retrieve the BGP from the LeBron James node previously explored by using the property "award received" (P166)

#final query

queryString = """
SELECT DISTINCT ?award ?awardName
WHERE {
   wd:Q36159 wdt:P166 ?award .
   ?award sc:name ?awardName 
   
   FILTER(REGEX(?awardName, "NBA Most Valuable"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)



Results
[('award', 'http://www.wikidata.org/entity/Q222047'), ('awardName', 'NBA Most Valuable Player Award')]
1


## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [44]:
#try this query but is wrong, it counts for every player 1 win of the MVP also if he won more than one

queryString = """
SELECT ?player ?playerName (COUNT(DISTINCT *) as ?victories)
WHERE {
         
   #get all the basketball players that won the NBA MVP award
   ?player wdt:P106 wd:Q3665646 ;
           wdt:P166 wd:Q222047 ;
           sc:name ?playerName . 
}
GROUP BY ?player ?playerName
LIMIT 100 
"""

print("Results")
x=run_query(queryString)




Results
[('player', 'http://www.wikidata.org/entity/Q8991894'), ('playerName', 'Giannis Antetokounmpo'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q29545'), ('playerName', 'Kevin Durant'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q312643'), ('playerName', 'Bob Cousy'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q190094'), ('playerName', 'Allen Iverson'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q44068'), ('playerName', 'Dirk Nowitzki'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q182455'), ('playerName', 'Wilt Chamberlain'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q199896'), ('playerName', 'Karl Malone'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q192707'), ('playerName', 'Charles Barkley'), ('victories', '1')]
[('player', 'http://www.wikidata.org/entity/Q242185'), ('playerName', 'Bill Russell'), ('victories', '1')]
[('player', 'htt

In [45]:
#explore the NBA MVP node by performing (1)

queryString = """
SELECT ?p ?pName ?obj ?oName
WHERE {
   wd:Q222047 ?p ?obj .
   FILTER(!isLiteral(?obj))
   
   ?p sc:name ?pName.
   ?obj sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q179051'), ('oName', 'Kareem Abdul-Jabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('oName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q134183'), ('oName', 'Magic Johnson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q17717'), ('oName', 'Oscar Robertson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q182455'), ('oName', 'Wilt Chamberlain')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'w

In [46]:
#explore the NBA MVP node by performing (2)

queryString = """
SELECT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q222047 .
   ?s sc:name ?sName .
   ?p sc:name ?pName .
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q352696'), ('sName', 'Michael Beasley'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nominated for')]
[('s', 'http://www.wikidata.org/entity/Q264393'), ('sName', 'Dave Cowens'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q190152'), ('sName', 'Larry Bird'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q553232'), ('sName', 'Bob McAdoo'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q273256'), ('sName', 'Hakeem Olajuwon'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q17717'), ('sName', 'Oscar Robertson'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('s', 'http://www.wikidata.org/entity/Q368561'

In [47]:
#explore the initial node by performing (3) 

queryString = """
SELECT DISTINCT ?p ?pName ?obj
WHERE {
   wd:Q222047 ?p ?obj .
   ?p sc:name ?pName.
   FILTER(isLiteral(?obj))
   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('pName', 'name in kana'), ('obj', 'さいゆうしゅうせんしゅしょう')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pName', 'inception'), ('obj', '1956-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('obj', '/m/02yqk8')]
3


In [48]:
#explore the winner property properties

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:P166 ?p ?obj .
   ?p sc:name ?pName.
   ?obj sc:name ?objName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q987080'), ('objName', 'honorary citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q618779'), ('objName', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q212190'), ('objName', 'Jehan Sadat')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q223566'), ('objName', 'Al Ahly SC')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q1325'), ('objName', 'Friedrich Hayek')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikid

In [49]:
#explore the winner property properties

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:P166 .
   ?s sc:name ?sName.
   ?p sc:name ?pName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)



Results
[('s', 'http://www.wikidata.org/entity/Q618779'), ('sName', 'award'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q66794791'), ('sName', 'by award or recognition received'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q987080'), ('sName', 'honorary citizenship'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q66796038'), ('sName', 'award received by'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q3320743'), ('sName', 'title of honor'), ('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property')]
[('s', 'http://www.wikidata.org/entity/Q82955'), ('sName', 'politician'), ('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for th

In [50]:
#explore the award received property properties

queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   wd:P166 ?p ?obj .
   ?p sc:name ?pName.
   ?obj sc:name ?objName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q987080'), ('objName', 'honorary citizenship')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('obj', 'http://www.wikidata.org/entity/Q618779'), ('objName', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q212190'), ('objName', 'Jehan Sadat')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q223566'), ('objName', 'Al Ahly SC')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikidata.org/entity/Q1325'), ('objName', 'Friedrich Hayek')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('obj', 'http://www.wikid

Final query for this task

In [51]:
#I couldn't manage to retrieve the date of every MVP award won by a player: I managed to find which player won the award at least one time, but I couldn't find 
#how to discriminate in which year every winning player won it. 
# I think that this is a property of the statement (reification) where the properties P166 (award received) or P1346 (winner) are involved. 
# This can be retrieved by using the qulifiers of Wikidata but I don't know if this is cheating. 